<a href="https://colab.research.google.com/github/mohamedbakhet/Medicine-In-Take-Reminder-and-Monitor/blob/master/ResNet_naive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow
! pip install h5py

In [ ]:
from keras.models import model_from_json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import numpy as np
import pandas as pd
from random import shuffle
import warnings
from keras.layers import Input
warnings.filterwarnings('ignore')
from keras.layers import Input
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import Sequential,models
from keras.models import Model
from tensorflow.keras import layers
from keras.layers import Input
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,MaxPooling2D,Conv1D,MaxPooling1D,add,Activation,GlobalAveragePooling2D,BatchNormalization,ReLU,MaxPool2D,SeparableConv2D,Add,GlobalAvgPool2D
#from keras.utils import plot_model
from keras.layers.merge import concatenate
from keras.utils.np_utils import to_categorical
import os,keras
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


Mounted at /content/drive


In [ ]:
#load train data set
training_data=[]
for index,value in enumerate (os.listdir('/content/drive/MyDrive/train_data')): #path to the data set on google drive
  print(value)
  lis=np.load('/content/drive/MyDrive/train_data/'+value)
  for j in lis[:1600]:
    training_data.append([np.array(j),value[:-4]])
shuffle(training_data)

#load test data set
test_data=[]
for index,value in enumerate (os.listdir('/content/drive/MyDrive/train_data')):
  print(value)
  lis=np.load('/content/drive/MyDrive/train_data/'+value)
  for j in lis[:400]:
    test_data.append([np.array(j),value[:-4]])
shuffle(test_data)

In [ ]:
#split data into train_test & encoding target

X_train = np.array([i[0] for i in training_data]).reshape(-1,300,15,1)
Y_train = np.array([i[1] for i in training_data])
train_labels_set = np.unique(Y_train,axis=0)
train_labels = LabelEncoder()
train_labels = train_labels.fit(Y_train)
Y_train = to_categorical(train_labels.transform(Y_train))

X_test = np.array([i[0] for i in test_data]).reshape(-1,300,15,1)
Y_test = np.array([i[1] for i in test_data])
test_labels_set = np.unique(Y_test,axis=0)
test_labels = LabelEncoder()
test_labels = test_labels.fit(Y_test)
Y_test = to_categorical(test_labels.transform(Y_test))

In [ ]:
#ResNet-naive
# function for creating an identity or projection residual module
def residual_module(layer_in, n_filters):
	merge_input = layer_in
	# check if the number of filters needs to be increase, assumes channels last format
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv1
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv2
	conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	# add filters, assumes filters/channels last
	layer_out = add([conv2, merge_input])
	# activation function
	layer_out = Activation('relu')(layer_out)
	return layer_out

# define model input
visible = Input(shape=(300, 15, 1))
# add vgg module
layer = residual_module(visible, 64)
flat_1 = Flatten()(layer)
den_1 = Dense(512,activation='relu')(flat_1)
den_1 = Dense(256,activation='relu')(den_1)
den_1 = Dense(128,activation='relu')(den_1)
den_1 = Dense(64,activation='relu')(den_1)
den_1 = Dense(32,activation='relu')(den_1)
out = Dense(14,activation='softmax')(den_1)
#den_1 = Dense()
# create model
model = Model(inputs=visible, outputs=out)

In [ ]:
#get the gpu name 
device_name = tf.test.gpu_device_name() #you can ignore this if don't have to use the gpu during the training proccess

In [ ]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
with tf.device(device_name): #in case that you skipped the gpu device comment these two lines and uncomment the last one and run
  hist = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test,Y_test))
#hist = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test,Y_test))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("ResNet-naive.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("ResNet-naive.h5")

In [ ]:
# load json and create model
json_file = open('ResNet-naive.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("ResNet-naive.h5")

In [ ]:
#to evaluate the model
opt = Adam(lr=0.001)
loaded_model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
y_predict = loaded_model.predict(X_test)
print(classification_report(Y_test.argmax(axis=1),y_predict.argmax(axis=1),target_names=test_labels_set))